In [18]:
import requests
from obi_auth import get_token
from entitysdk import Client, ProjectContext, models
from enum import Enum

class DatabaseEnvironment(Enum):
    STAGING_DATABASE = "staging"
    # PRODUCTION_DATABASE = "production"
    # LOCAL_UNAUTHENTICATED = "local_unauthenticated"
    # Note: LOCAL_AUTHENTICATED is not supported by entitycore because the project/vlab ids do not exist in your keycloak groups

obi_one_api_url = "http://127.0.0.1:8100"
# obi_one_api_url = "https://staging.openbraininstitute.org/api/obi-one"

add_morphology_asset = False

database_environment = DatabaseEnvironment.STAGING_DATABASE
# database_environment = DatabaseEnvironment.PRODUCTION_DATABASE
# database_environment = DatabaseEnvironment.LOCAL_UNAUTHENTICATED

if database_environment == DatabaseEnvironment.STAGING_DATABASE:
    """
    - Uncomment in obi-one/.env.run-local before "make run-local": 
        export ENTITYCORE_URL=https://staging.openbraininstitute.org/api/entitycore
    """
    entitycore_api_url = "https://staging.openbraininstitute.org/api/entitycore"
    virtual_lab_id="d6182f2e-ca09-4293-91d4-592b9e764ef7"
    project_id="b5484454-1c42-49e6-89fb-36e89109a2e1"

# elif database_environment == DatabaseEnvironment.PRODUCTION_DATABASE:
#     """
#     For future: EntityCore not currently in production.
#     - Uncomment in obi-one/.env.run-local before "make run-local": 
#         export ENTITYCORE_URL=https://www.openbraininstitute.org/api/entitycore
#     """
#     entitycore_api_url = "https://www.openbraininstitute.org/api/entitycore"

elif database_environment == DatabaseEnvironment.LOCAL_UNAUTHENTICATED:
    """
    Not yet tested.
    - Launch entitycore locally (make run-local)
    - Add a morphology with a project name (see entitysdk)
    """
    entitycore_api_url = "http://127.0.0.1:8000"
    add_morphology_asset = True
    virtual_lab_id="a98b7abc-fc46-4700-9e3d-37137812c730"
    project_id="0dbced5f-cc3d-488a-8c7f-cfb8ea039dc6"
    # virtual_lab_id="e6030ed8-a589-4be2-80a6-f975406eb1f6"
    # project_id="2720f785-a3a2-4472-969d-19a53891c817"
    
else:
    raise ValueError(f"Unsupported environment: {database_environment}")


token = get_token(environment="staging")
project_context = ProjectContext(virtual_lab_id=virtual_lab_id, project_id=project_id)
client = Client(api_url=entitycore_api_url, project_context=project_context, token_manager=token)

[2025-07-07 16:08:35,551] INFO: Local server listening on http://localhost:49838
[2025-07-07 16:08:35,552] INFO: Authentication url: https://staging.openbraininstitute.org/auth/realms/SBO/protocol/openid-connect/auth?response_type=code&client_id=obi-entitysdk-auth&redirect_uri=http%3A%2F%2Flocalhost%3A49838%2Fcallback&scope=openid&code_challenge=drNFg_zF7XNR6fpuVayf6zFqWrjhNZOa2Afpa6QGrbc&code_challenge_method=S256&kc_idp_hint=github
[2025-07-07 16:08:37,548] INFO: HTTP Request: POST https://staging.openbraininstitute.org/auth/realms/SBO/protocol/openid-connect/token "HTTP/1.1 200 OK"


# Circuit ID

In [19]:
circuit_id = "3d0dcdfd-0534-49e6-aeea-d78d35948c7f"

# Create SimulationForm using Cirucit

In [20]:
import obi_one as obi

# Sim duration
sim_duration = 1000.0

# Empty Simulation Configuration
sim_conf = obi.SimulationsForm.empty_config()

# Info
info = obi.Info(campaign_name="O1 Simulation", campaign_description="Simulation of O1 circuit with predefined neuron set and constant current stimulus")
sim_conf.set(info, name="info")

# Neuron Sets
sim_neuron_set = obi.IDNeuronSet(neuron_ids=obi.NamedTuple(name="IDNeuronSet1", elements=range(10)))
sim_conf.add(sim_neuron_set, name='L1All')

all_neuron_set = obi.AllNeurons()
sim_conf.add(all_neuron_set, name='AllNeurons')

sync_neuron_set = obi.IDNeuronSet(neuron_ids=obi.NamedTuple(name="IDNeuronSet2", elements=range(3)))
sim_conf.add(sync_neuron_set, name='ID3')

vpm_neuron_set = obi.nbS1VPMInputs(sample_percentage=100)
sim_conf.add(vpm_neuron_set, name='nbS1VPMInputs')

# Regular Timesteps
regular_timestamps = obi.RegularTimestamps(start_time=0.0, number_of_repetitions=3, interval=sim_duration)
sim_conf.add(regular_timestamps, name='RegularTimestamps')

# Stimulus
poisson_input = obi.PoissonSpikeStimulus(duration=800.0, timestamps=regular_timestamps.ref, frequency=20, source_neuron_set=sim_neuron_set.ref, targeted_neuron_set=sim_neuron_set.ref)
sim_conf.add(poisson_input, name='PoissonInputStimulus')

sync_input = obi.FullySynchronousSpikeStimulus(timestamps=regular_timestamps.ref, source_neuron_set=sync_neuron_set.ref, targeted_neuron_set=sim_neuron_set.ref)
sim_conf.add(sync_input, name='SynchronousInputStimulus')

current_stim = obi.ConstantCurrentClampSomaticStimulus(timestamps=regular_timestamps.ref, neuron_set=sim_neuron_set.ref, amplitude=0.1)
sim_conf.add(current_stim, name='ConstantCurrentClampSomaticStimulus')

# Recordings
voltage_recording = obi.SomaVoltageRecording(neuron_set=sim_neuron_set.ref)
sim_conf.add(voltage_recording, name='VoltageRecording')

time_window_voltage_recording = obi.TimeWindowSomaVoltageRecording(neuron_set=sim_neuron_set.ref, start_time=0.0, end_time=2000.0)
sim_conf.add(time_window_voltage_recording, name='TimeWindowVoltageRecording')

# Initialization
simulations_initialize = obi.SimulationsForm.Initialize(circuit=obi.CircuitFromID(id_str=circuit_id), 
                                                        node_set=sim_neuron_set.ref, 
                                                        simulation_length=sim_duration)
sim_conf.set(simulations_initialize, name='initialize')

# Validated Config
validated_sim_conf = sim_conf.validated_config()

# Generate GridScan (locally) - Note strange issue described for relative paths

In [21]:
import tempfile

# Create a temporary directory (not used in this example, but can be useful for testing)
with tempfile.TemporaryDirectory() as temp_dir:

    """
    Very strange issue with adding assets when a relative path is used here. Need to look into after CNS.
    """
    # grid_scan = obi.GridScan(form=validated_sim_conf, coordinate_directory_option="ZERO_INDEX", output_root='../../obi-output/circuit_simulations/grid_scan')
    grid_scan = obi.GridScan(form=validated_sim_conf, coordinate_directory_option="ZERO_INDEX", output_root='/Users/james/Documents/obi/code/obi-output/circuit_simulations/grid_scan')
    # grid_scan = obi.GridScan(form=validated_sim_conf, coordinate_directory_option="ZERO_INDEX", output_root=temp_dir)
    grid_scan.multiple_value_parameters(display=True)
    grid_scan.coordinate_parameters(display=True)
    grid_scan.execute(processing_method='generate', data_postprocessing_method='save', db_client=client)

[2025-07-07 16:09:17,835] INFO: 
MULTIPLE VALUE PARAMETERS
[2025-07-07 16:09:17,835] INFO: No multiple value parameters found.
[2025-07-07 16:09:17,836] INFO: 
COORDINATE PARAMETERS
[2025-07-07 16:09:17,836] INFO: No coordinate parameters.
[2025-07-07 16:09:17,836] INFO: <entitysdk.client.Client object at 0x3185546e0>
[2025-07-07 16:09:17,838] INFO: create_bbp_workflow_campaign_config() not yet complete.
[2025-07-07 16:09:17,839] INFO: 1. Initializing simulation campaign in the database...
[2025-07-07 16:09:17,839] INFO: -- Register SimulationCampaign Entity
[2025-07-07 16:09:18,319] INFO: HTTP Request: POST https://staging.openbraininstitute.org/api/entitycore/simulation-campaign "HTTP/1.1 403 Forbidden"


EntitySDKError: HTTP error 403 for POST https://staging.openbraininstitute.org/api/entitycore/simulation-campaign
data       : None
json       : {
  "name": "O1 Simulation",
  "description": "Simulation of O1 circuit with predefined neuron set and constant current stimulus",
  "type": null,
  "created_by": null,
  "updated_by": null,
  "authorized_public": false,
  "authorized_project_id": null,
  "contributions": null,
  "legacy_id": null,
  "scan_parameters": {},
  "entity_id": "3d0dcdfd-0534-49e6-aeea-d78d35948c7f",
  "simulations": null
}
params     : None
response   : {"error_code":"NOT_AUTHORIZED","message":"User not authorized for the given virtual-lab-id or project-id","details":null}

# Test endpoint. Service must be launched first (make run-local)

In [22]:
import requests

# Construct the full endpoint URL
url = f"{obi_one_api_url}/generated/simulations-generate-grid-save"
# Prepare headers
headers = {
    "Authorization": f"Bearer {token}",
    "Accept": "application/json",
    "Content-Type": "application/json"
}

# Optionally include optional headers if they are set
if virtual_lab_id:
    headers["virtual-lab-id"] = virtual_lab_id
if project_id:
    headers["project-id"] = project_id

# Construct request
# Placeholder example:
request_body = {
 'type': 'SimulationsForm',
 'timestamps': {'RegularTimestamps': {'type': 'RegularTimestamps',
   'start_time': 0.0,
   'number_of_repetitions': 3,
   'interval': 3000.0}},
 'stimuli': {
  'PoissonInputStimulus': {'type': 'PoissonSpikeStimulus',
   'timestamps': {'type': 'TimestampsReference',
    'block_dict_name': 'timestamps',
    'block_name': 'RegularTimestamps'},
   'source_neuron_set': {'type': 'NeuronSetReference',
    'block_dict_name': 'neuron_sets',
    'block_name': 'L1All'},
   'targeted_neuron_set': {'type': 'NeuronSetReference',
    'block_dict_name': 'neuron_sets',
    'block_name': 'L1All'},
   'duration': 800.0,
   'random_sample': 0,
   'frequency': 20.0}
   },
 'recordings': {'VoltageRecording': {'type': 'SomaVoltageRecording',
   'start_time': 0.0,
   'end_time': 3000.0,
   'dt': 0.1,
   'neuron_set': {'type': 'NeuronSetReference',
    'block_dict_name': 'neuron_sets',
    'block_name': 'L1All'}}},
 'neuron_sets': {'L1All': {'type': 'IDNeuronSet',
    'sample_percentage': 100,
   'sample_seed': 0,
   'neuron_ids': {'type': 'NamedTuple',
    'name': 'IDNeuronSet1',
    'elements': (0, 1, 2, 3, 4, 5, 6, 7, 8, 9)}}},
 'initialize': {'type': 'SimulationsForm.Initialize',
  'circuit': {'type': 'CircuitFromID',
   'id_str': circuit_id},
  'simulation_length': 3000.0,
  'node_set': {'type': 'NeuronSetReference',
   'block_dict_name': 'neuron_sets',
   'block_name': 'L1All'},
  'sample_seed': 1,
  'extracellular_calcium_concentration': 1.1,
  'v_init': -80.0},
 'info': {'type': 'Info',
  'campaign_name': 'No name provided',
  'campaign_description': 'No description provided'}
}

# Make the POST request
response = requests.post(url, headers=headers, json=request_body)

# Check the response
if response.status_code == 200:
    data = response.json()
    simulation_campaign_id = data
    print("Success:", data)
else:
    print(f"Error {response.status_code}: {response.text}")

Error 403: {"error_code":"NOT_AUTHORIZED","message":"User not authorized for the given virtual-lab-id or project-id","details":null}


# Stage the Circuit

In [ ]:
from entitysdk.models import Circuit
circuit = client.get_entity(entity_id=circuit_id, entity_type=Circuit)

from entitysdk.staging.circuit import stage_circuit
from pathlib import Path
stage_circuit(client=client, model=circuit, output_dir=Path('temp_circ/'))

# Stage the Simulation

In [ ]:
from entitysdk.models import SimulationCampaign, Simulation
simulation_campaign = client.get_entity(entity_id=simulation_campaign_id, entity_type=SimulationCampaign)
simulation = client.get_entity(entity_id=simulation_campaign.simulations[0].id, entity_type=Simulation)

for asset in simulation.assets:
    if asset.content_type == "application/json":
        if asset.label == 'sonata_simulation_config':
            # Download the content into memory
            content = client.download_file(
                entity_id=simulation.id,
                entity_type=Simulation,
                asset_id=asset.id,
                output_path='simulation_config.json'
            )

from pathlib import Path
from entitysdk.staging.simulation import stage_simulation
stage_simulation(client=client, model=simulation, output_dir=Path('temp_sim'), circuit_config_path=Path("../temp_circ/circuit_config.json"))

# For testing (please ignore)

In [ ]:
# import entitysdk
# import tempfile
# from pathlib import Path

# from entitysdk.models import SimulationCampaign, Simulation

# _campaign = client.register_entity(
#     entitysdk.models.SimulationCampaign(
#         name="Hello",
#         description="World",
#         entity_id="662dce5c-306d-4737-acd6-a3af985aad7f",
#         scan_parameters={},
#     )
# )

# simulation = client.register_entity(
#     entitysdk.models.Simulation(
#         name="sim-0",
#         description="sim-0",
#         scan_parameters={"foo": "bar"},
#         entity_id="662dce5c-306d-4737-acd6-a3af985aad7f",
#         simulation_campaign_id=_campaign.id
#     )
# )

# _ = client.upload_file(
#     entity_id=simulation.id,
#     entity_type=entitysdk.models.Simulation,
#     file_path=Path("simulation_config.json"),
#     file_content_type="application/json",
#     asset_label='sonata_simulation_config'
# )

# with tempfile.TemporaryDirectory() as tdir:
#     file1 = Path(tdir, "poisson.h5")
#     file1.write_text("hello")
#     _ = client.upload_file(
#                 entity_id=simulation.id,
#                 entity_type=entitysdk.models.Simulation,
#                 file_path=file1,
#                 file_content_type="application/x-hdf5",
#                 asset_label='replay_spikes'
#             )
    
# simulation = client.get_entity(entity_id=simulation_campaign.simulations[0].id, entity_type=Simulation)
# print(simulation)

# from pathlib import Path
# from entitysdk.staging.simulation import stage_simulation
# stage_simulation(client=client, model=simulation, output_dir=Path('temp_sim'), circuit_config_path=Path("temp_circ/circuit_config.json"))

# Trying to reproduce relative path error (please ignore)

In [ ]:
# import entitysdk

# _campaign = client.register_entity(
#             entitysdk.models.SimulationCampaign(
#                 name="Hello",
#                 description="World",
#                 entity_id="662dce5c-306d-4737-acd6-a3af985aad7f",
#                 scan_parameters={},
#             )
#         )

# from pathlib import Path

# model_dump = {"foo": "bar", "baz": 42}
# import json



# with tempfile.TemporaryDirectory() as tdir:

#     output_path = Path("../run_scan_config.json")
#     # output_path = Path("/Users/james/Documents/obi/code/obi-one/obi-output/run_scan_config.json")
#     # output_path = Path(tdir, "run_scan_config.json")

#     output_path.write_text("swc")

#     _ = client.upload_file(
#         entity_id=_campaign.id,
#         entity_type=entitysdk.models.SimulationCampaign,
#         file_path=output_path,
#         file_content_type="application/json",
#         asset_label='campaign_generation_config'
#     )